# web scrapping to extract reviews of my laptop from amazon website 

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
amazon_url = 'https://www.amazon.in/HP-Pavilion-Processor-Graphics-15-ec2150AX/product-reviews/B09MF8KMRW/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=1'

In [6]:
# writing a function for requesting amazon for the data and parsing the data using Beautiful soup

In [7]:
 header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/'}
review_page = requests.get(url = amazon_url ,headers=header)
soup = BeautifulSoup(review_page.text , 'html.parser')

In [8]:
def get_soup(amazon_url):
    header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/'}
    review_page = requests.get(url = amazon_url ,headers=header)
    soup = BeautifulSoup(review_page.text , 'html.parser')
    return soup

In [9]:
review_list = []

In [10]:
def get_reviews(soup):
    reviews = soup.find_all('div',{'data-hook':'review'})
    try:
        for item in reviews:
            review = {'comment': item.find('span',{'data-hook':'review-body'}).text.strip()}
            review_list.append(review)
    except:
        pass


In [ ]:
for x in range(1,10):
    a = get_soup(f'https://www.amazon.in/HP-Pavilion-Processor-Graphics-15-ec2150AX/product-reviews/B09MF8KMRW/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber={x}')
    print(f'Getting page {x}')
    get_reviews(a)
    print(len(review_list))
    if not soup.find('li',{'class':'a-disabled a-last'}):
        pass
    else:
        break
    

In [ ]:
import pandas as pd
pavillion_reviews = pd.DataFrame(review_list)

In [ ]:
pavillion_reviews.to_csv('amazon')

In [ ]:
pavillion_reviews.isna().sum()

# textmining

In [ ]:
import numpy as np
import pandas as pd
import string 
import spacy 
import re 
from matplotlib.pyplot import imread
from matplotlib import pyplot as plt
from wordcloud import WordCloud
%matplotlib inline 

In [ ]:
pavillion_reviews.comment = pavillion_reviews.comment.apply(lambda x:re.sub('[^a-zA-Z]',' ',x))
pavillion_reviews

In [ ]:
pavillion_reviews_copy = pavillion_reviews.copy()

In [ ]:
pavillion_reviews = [comment.strip() for comment in pavillion_reviews.comment] # remove both the leading and the trailing characters
pavillion_reviews = [comment for comment in pavillion_reviews if comment] # removes empty strings, because they are considered in Python as False
pavillion_reviews[0:10] 

In [ ]:
# Joining the list into one string/text
text = ' '.join(pavillion_reviews)
text 

In [ ]:
#Punctuation
no_punc_text = text.translate(str.maketrans('', '', string.punctuation))
no_punc_text

In [ ]:
#Tokenization
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
text_tokens = word_tokenize(no_punc_text)
print(text_tokens[0:50]) 

In [ ]:
len(text_tokens) 

In [ ]:
#Remove stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

my_stop_words = stopwords.words('english')
my_stop_words.append(['the','and'])
no_stop_tokens = [word for word in text_tokens if not word in my_stop_words]
print(no_stop_tokens[0:40]) 

In [ ]:
#Noramalize the data
lower_words = [x.lower() for x in no_stop_tokens]
print(lower_words[0:25]) 

In [ ]:
# NLP english language model of spacy library
nlp = spacy.load('en_core_web_sm')   

In [ ]:
# lemmas being one of them, but mostly POS, which will follow later
doc = nlp(' '.join(no_stop_tokens))
print(doc[0:40]) 

In [ ]:
lemmas = [token.lemma_ for token in doc]
print(lemmas[0:25])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_n_gram_max_features = TfidfVectorizer(norm="l2",analyzer='word', ngram_range=(1,3), max_features = 500)
tf_idf_matrix_n_gram_max_features =vectorizer_n_gram_max_features.fit_transform(pavillion_reviews)
print(vectorizer_n_gram_max_features.get_feature_names())
print(tf_idf_matrix_n_gram_max_features.toarray()) 

In [ ]:
# Import packages
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS
# Define a function to plot word cloud
def plot_cloud(wordcloud):
    # Set figure size
    plt.figure(figsize=(40, 30))
    # Display image
    plt.imshow(wordcloud) 
    # No axis details
    plt.axis("off"); 

In [ ]:
# Generate wordcloud
stopwords = STOPWORDS
stopwords.add('will')
wordcloud = WordCloud(width = 3000, height = 2000, background_color='black', max_words=100,colormap='Set2',stopwords=stopwords).generate(text)
# Plot
plot_cloud(wordcloud)

# emotional mining

In [ ]:
# sentiment analysis using vader

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
sentiment =  SentimentIntensityAnalyzer()

In [ ]:
sentiment_dict = sentiment.polarity_scores(text)
sentiment_dict

In [ ]:
# sentiment analysis using textblob

In [ ]:
from textblob import TextBlob

In [ ]:
#Create a function to get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

In [ ]:
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [ ]:
#Create two new columns ‘Subjectivity’ & ‘Polarity’
pavillion_reviews_copy['Subjectivity'] = pavillion_reviews_copy['comment'].apply(getSubjectivity)
pavillion_reviews_copy['Polarity'] = pavillion_reviews_copy['comment'].apply(getPolarity)

In [ ]:
pavillion_reviews_copy

In [ ]:
def getAnalysis(score):
        if score < 0:
            return 'Negative'
        elif score == 0:
            return 'Neutral'
        else:
            return 'Positive'
        return TextBlob(text).apply(getAnalysis)

In [ ]:
pavillion_reviews_copy['Analysis'] = pavillion_reviews_copy['Polarity'].apply(getAnalysis )
pavillion_reviews_copy

In [ ]:
pavillion_reviews_copy['Analysis'].value_counts().plot(kind = 'bar')
# most of the reviews are positive in nature

In [ ]:
# Sentiment score of the whole reviews
pavillion_reviews_copy.describe().T

In [ ]:
# as subjectivity greater 0 suggests the reviews are opinions 

In [ ]:
# positive reviews 
pavillion_reviews_copy[pavillion_reviews_copy['Analysis']=='Positive']

In [ ]:
# negative reviews 
pavillion_reviews_copy[pavillion_reviews_copy['Analysis']=='Negative']

In [ ]:
# neutral reviews 
pavillion_reviews_copy[pavillion_reviews_copy['Analysis']=='Neutral']